# Deep Deterministic Policy Gradients (DDPG)
---
In this notebook, we train DDPG with OpenAI Gym's BipedalWalker-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import numpy as np
import random
import copy
from collections import namedtuple, deque
from replay_buffer import ReplayBuffer

import tensorflow as tf
from keras import backend as K
from keras.layers import Dense, Input, Add, Activation,LeakyReLU
from keras.layers import GaussianNoise, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent
from ClusterWorld import ClusterWorldEnv
from infoCalc import *

Using TensorFlow backend.


### 2. Instantiate the Environment and Agent

In [2]:
sess = tf.Session()
K.set_session(sess)

env = ClusterWorldEnv()
env.seed(10)
agent = Agent(env, sess,state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], random_seed=10)
agent.NetworkSummary()
#agent.load_network('savedModels/', 'saved.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Actor Summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 16)           80          input_2[0][0]                    
__________________________________________________________________________________________________
gaussian_noise_3 (GaussianNoise (None, 16)           0           dense_5[0][0]                    
__________________________________________________________________________________________________
activat

### 3. Train the Agent with DDPG

Run the code cell below to train the agent from scratch.  Alternatively, you can skip to the next code cell to load the pre-trained weights from file.

In [3]:
def ddpg(n_episodes=10000, max_t=700):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent.reset()

        score = 0
        for t in range(max_t):
            action = agent.act(state,True)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 100 == 0:
            agent.save_network('savedModels/', 'saved.h5')
            print(action)
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
        agent.save_network('savedModels/', 'saved.h5')
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 64	Average Score: -100.00	Score: -100.00WARNING:tensorflow:From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Episode 100	Average Score: -100.00	Score: -100.00[0.33361718 0.13080052 0.85386765 1.        ]
Episode 100	Average Score: -100.00
Episode 200	Average Score: -100.00	Score: -100.00[0.38708064 0.2381483  0.8129973  1.        ]
Episode 200	Average Score: -100.00
Episode 300	Average Score: -100.00	Score: -100.00[0.5434507  0.07948818 0.6078654  1.        ]
Episode 300	Average Score: -100.00
Episode 400	Average Score: -310.17	Score: -700.00[1.         0.64066833 0.71504337 0.        ]
Episode 400	Average Score: -310.17
Episode 420	Average Score: -400.18	Score: -700.00

KeyboardInterrupt: 

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
agent.load_network('savedModels/', 'saved.h5')
state = env.reset()
   
while True:
    action = agent.act(state,False)
    env.render()
    next_state, reward, done, _ = env.step(action)
    state = next_state
    if done:
        break
        


### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  